In [1]:
import pandas as pd
import pubchempy as pcp
import concurrent.futures
import os

# Chemin vers ton fichier CSV
input_csv = "photocompounds_database.csv"
output_csv = "photocompounds_database_pubchem.csv"

# Charger le fichier CSV
data = pd.read_csv(input_csv, sep=";")

In [2]:
# Colonnes à ajouter
new_columns = ["Molecular Weight", "Chemical Formula", "SMILES", 
               "H-Bond Donor Count", "H-Bond Acceptor Count", 
               "Solubility", "Melting Point", "Boiling Point", "pKa"]

# Ajouter les nouvelles colonnes si elles n'existent pas encore
for col in new_columns:
    if col not in data.columns:
        data[col] = ""

In [3]:
# Fonction pour enrichir les données pour un CAS donné
def enrich_row(index, row):
    cas_number = row["CAS"]
    try:
        # Vérifier si les données sont déjà présentes
        if pd.notna(row["SMILES"]):
            return row  # Déjà enrichi, on passe

        # Récupérer les informations depuis PubChem
        compound = pcp.get_compounds(cas_number, "name")
        if compound:
            comp = compound[0]
            row["Molecular Weight"] = comp.molecular_weight
            row["Chemical Formula"] = comp.molecular_formula
            row["SMILES"] = comp.canonical_smiles
            row["H-Bond Donor Count"] = comp.h_bond_donor_count
            row["H-Bond Acceptor Count"] = comp.h_bond_acceptor_count
            
            # Extraire les propriétés supplémentaires
            properties = comp.to_dict()
            row["Solubility"] = properties.get("Solubility", "")
            row["Melting Point"] = properties.get("Melting Point", "")
            row["Boiling Point"] = properties.get("Boiling Point", "")
            row["pKa"] = properties.get("pKa", "")

            print(f"✅ Informations ajoutées pour le CAS {cas_number}")
        else:
            print(f"⚠️ Aucune information trouvée pour le CAS {cas_number}")

    except Exception as e:
        print(f"⚠️ Erreur pour le CAS {cas_number} : {e}")

    return row

In [4]:
# Fonction pour enrichir les données pour un CAS donné
def enrich_row(index, row):
    cas_number = row["CAS"]
    try:
        # Vérifier si les données sont déjà présentes
        if pd.notna(row["SMILES"]):
            return row  # Déjà enrichi, on passe

        # Récupérer les informations depuis PubChem
        compound = pcp.get_compounds(cas_number, "name")
        if compound:
            comp = compound[0]
            row["Molecular Weight"] = comp.molecular_weight
            row["Chemical Formula"] = comp.molecular_formula
            row["SMILES"] = comp.canonical_smiles
            row["H-Bond Donor Count"] = comp.h_bond_donor_count
            row["H-Bond Acceptor Count"] = comp.h_bond_acceptor_count
            
            # Extraire les propriétés supplémentaires
            properties = comp.to_dict()
            row["Solubility"] = properties.get("Solubility", "")
            row["Melting Point"] = properties.get("Melting Point", "")
            row["Boiling Point"] = properties.get("Boiling Point", "")
            row["pKa"] = properties.get("pKa", "")

            print(f"✅ Informations ajoutées pour le CAS {cas_number}")
        else:
            print(f"⚠️ Aucune information trouvée pour le CAS {cas_number}")

    except Exception as e:
        print(f"⚠️ Erreur pour le CAS {cas_number} : {e}")

    return row

    # Utiliser un ThreadPoolExecutor pour accélérer le traitement
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda args: enrich_row(*args), data.iterrows()))

In [5]:
# Créer un nouveau DataFrame à partir des résultats
enriched_data = pd.DataFrame(results)

In [6]:
# Sauvegarder le fichier enrichi
enriched_data.to_csv(output_csv, sep=",", index=False)
print("\n✅ Fichier enrichi sauvegardé sous :", output_csv)


✅ Fichier enrichi sauvegardé sous : photocompounds_database_pubchem.csv
